<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instal and Setup (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-dbmjnviy
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-dbmjnviy
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 3464467af44286e8657f1a0a42257831939abe82
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 663.9 kB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42392 sha256=34c32e62992d2be0f35bf2fadaa1641464ad0959e3fac44ebcee2b5ce45c9337
  Stored in directory: /tmp/pip-ephem-wheel-cache-c3v6y7ze/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61

In [2]:
# @title Install Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

#### Setup and Configuration

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        url = f'{self.api_url}/{dir_path}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    dir='ibd'
)

In [5]:
# @title DataFrame Operations
import yfinance as yf
import vistock.tw as tw

def yf_get_current_price(ticker):
    stock = yf.Ticker(ticker)
    historical_data = stock.history(period="1d")
    return historical_data['Close'].iloc[-1]

def is_taiwan_stock_df(df):
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    return ticker.isdigit()

def add_price_column(df):
    column_names = df.columns.tolist()
    if 'Price' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    df['Price'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Price')
    df = df[column_names]
    if is_taiwan_stock_df(df):
        df['Price'] = df['Ticker'].apply(tw.stock_price)
    else:
        df['Price'] = df['Ticker'].apply(yf_get_current_price)
    return df

def add_name_column(df):
    column_names = df.columns.tolist()
    if 'Name' in column_names:
        return df
    if 'Ticker' not in column_names:
        return df
    ticker = df['Ticker'].iloc[0].replace('.TWO', '').replace('.TW', '')
    if not ticker.isdigit():    # if not a Taiwan stock
        return df
    df['Name'] = None
    ticker_index = column_names.index('Ticker')
    column_names.insert(ticker_index + 1, 'Name')
    df = df[column_names]
    df['Name'] = df['Ticker'].apply(tw.stock_name)
    return df

def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]

### Glossary of Terms

source (Select the source of stocks to analyze):
- The source of stocks to analyze
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange
    - `TPEX`: Taipei Exchange
    - `ESB`: Emerging Stock Board
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500
    - `DJIA`: Dow Jones Industrial Average
    - `NDX`: NASDAQ-100
    - `SOX`: PHLX Semiconductor Index
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.


### Execute Actions Step by Step


In [6]:
# @title Step 1. Pick a File

import ipywidgets as widgets
from IPython.display import display

# List of filenames
filenames = github.list_filenames()

# Extract and sort all unique values
dates = sorted(set(fn.split('_')[3].replace('.csv', '') for fn in filenames), reverse=True)
sources = sorted(set(fn.split('_')[0] for fn in filenames), reverse=True)
types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames for setting dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px per character for width

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create the file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

# Function to update file options based on selected filters
def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value in fn and
            type_dropdown.value in fn and
            period_dropdown.value in fn)
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match

# Function to handle file selection and print the selected filename
#def on_file_selected(change):
#    selected_file = change['new']
#    print(f"Selected file: {selected_file}")

# Update file options when any of the dropdowns change
date_dropdown.observe(update_file_options, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Print the selected file when it changes
#file_dropdown.observe(on_file_selected, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown, file_dropdown)

# Initialize the file dropdown with options
update_file_options()


Dropdown(description='Date:', layout=Layout(width='430px'), options=('20240812', '20240811', '20240809'), valu…

Dropdown(description='Source:', layout=Layout(width='430px'), options=('TWSE+TPEX', 'SPX+DJIA+NDX+SOX'), value…

Dropdown(description='Type:', layout=Layout(width='430px'), options=('stocks', 'industries'), value='stocks')

Dropdown(description='Period:', layout=Layout(width='430px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='430px'), options=(), value=None)

In [12]:
# @title Step 2. Print Top Percentile Items {"run":"auto"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
df_top = df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: SPX+DJIA+NDX+SOX
Type: stocks
Period: 2y
Date: 20240812


,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
0,NVDA,109.019997,Technology,Semiconductors,155.48,182.62,171.06,169.86,99,99,99,99,1
1,VST,77.959999,Utilities,Utilities - Independent Power Producers,152.96,180.25,249.59,143.40,99,99,99,98,2
2,IRM,108.669998,Real Estate,REIT - Specialty,145.88,126.15,117.71,104.93,99,95,90,72,3
3,FICO,1720.000000,Technology,Software - Application,143.91,130.81,117.93,128.72,99,98,91,96,4
4,GDDY,158.460007,Technology,Software - Infrastructure,140.42,128.66,135.09,118.19,99,97,97,92,5
5,HWM,92.830002,Industrials,Aerospace & Defense,138.34,121.55,134.77,115.23,99,94,97,88,6
6,NRG,79.870003,Utilities,Utilities - Independent Power Producers,136.27,127.20,172.97,118.22,98,96,99,92,7
7,MMM,123.360001,Industrials,Conglomerates,133.12,103.09,112.81,82.41,98,77,86,12,8
8,GE,166.869995,Industrials,Aerospace & Defense,131.27,121.63,144.67,122.44,98,94,98,94,9
9,TSM,167.630005,Technology,Semiconductors,130.89,139.94,131.16,116.99,98,99,96,91,10


['NVDA', 'VST', 'IRM', 'FICO', 'GDDY', 'HWM', 'NRG', 'MMM', 'GE', 'TSM']


In [10]:
# @title Step 3. Filter Items with Increasing RS > 100
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

selected_file = file_dropdown.value
source, kind, period, date = selected_file.split('_')
date = date.replace('.csv', '')
print(f"Source: {source}\nType: {kind}\nPeriod: {period}\nDate: {date}")

df = github.download_csv(selected_file)
filtered_df = filter_increasing_relative_strength(df)
df_top = filtered_df.head(num_items).copy()
df_top = add_price_column(df_top)
df_top = add_name_column(df_top)
display(df_top)

if 'Name' in df_top.columns:
    print(list(df_top['Name']))
if 'Ticker' in df_top.columns:
    print(list(df_top['Ticker']))

Source: SPX+DJIA+NDX+SOX
Type: stocks
Period: 2y
Date: 20240812


,Ticker,Price,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
2,IRM,108.669998,Real Estate,REIT - Specialty,145.88,126.15,117.71,104.93,99,95,90,72,3
11,TRGP,137.589996,Energy,Oil & Gas Midstream,128.78,122.07,120.33,105.88,97,94,92,73,12
24,VTR,58.980000,Real Estate,REIT - Healthcare Facilities,119.76,101.22,96.23,84.46,95,72,47,17,25
25,TYL,579.760010,Technology,Software - Application,119.69,106.76,102.08,97.91,95,85,64,53,26
31,WELL,116.620003,Real Estate,REIT—Healthcare Facilities,116.73,102.30,100.22,98.80,94,74,59,55,32
37,MSI,412.760010,Technology,Communication Equipment,115.27,106.75,104.27,96.33,92,85,69,50,38
45,EXR,164.110001,Real Estate,REIT - Industrial,114.48,99.75,99.60,91.21,91,69,57,37,46
46,AFL,101.190002,Financial Services,Insurance - Life,114.45,102.15,100.73,93.78,91,74,61,43,47
54,PM,116.089996,Consumer Defensive,Tobacco,113.56,98.26,97.50,85.60,89,67,50,20,55
55,MCO,460.579987,Financial Services,Financial Data & Stock Exchanges,113.36,105.41,103.53,99.70,89,82,68,58,56


['IRM', 'TRGP', 'VTR', 'TYL', 'WELL', 'MSI', 'EXR', 'AFL', 'PM', 'MCO']
